# Data Analysis for Smartphone Web Surveys: A Systematic Literature Review

This notebook uses the data obtained in the systematic literature review and creates a visualisation that supports the article's research questions. The notebook is structured as follows. First, the data is imported and preprocessed. After that, we create the plots and calculation for the evaluation of the systematic literature review. Then we create the visualisations for the article: "Smartphone Web Surveys: A Systematic Literature Review".

## Import and merge data

In [ ]:
import re

import pandas as pd
import geopandas as gpd
from matplotlib import cm
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable

from src.visualization.visualize import create_visualisation_data
from src.features.build_features import find_columns, check_columns, find_citation_key_topics

In [ ]:
# Import meta data from Citations
metadata_raw = pd.read_csv("../data/raw/literature_review_meta_data.csv")

In [ ]:
# Preprocess meta data
metadata_columns_include = ["Item Type", "Publication Year", "Author", "Title", "Publication Title",
                            "Conference Name"]
metadata = metadata_raw[metadata_columns_include]
metadata_column_name = ['article_genre', 'article_year', 'article_author', 'article_name', 'article_journal', 
                        'article_proceeding']
metadata.columns = metadata_column_name

In [ ]:
# Create citation key from meta data
main_author = metadata.article_author.str.split().str.get(0).str.lower().str.replace(',','')
first_word_title = metadata.article_name.str.split().str.get(0).str.lower()
article_year = metadata.article_year.astype(str)
metadata["citation_key"] = main_author + "_" + first_word_title +  "_" + article_year

In [ ]:
# Import coding results from literature review
coding_results_raw = pd.read_excel("../data/raw/literature_coding_results.xlsx")

In [ ]:
# Merge data
data = pd.merge(metadata, coding_results_raw, how = "left", on='article_name')

In [ ]:
# Save data
data.to_csv("../data/interim/combined_data.csv", index = False)

## Analyze metadata

In this section of the notebook we analyse the extracted meta data from the literature review, that will be used for the analyses in section 3.1 in the paper.

### Overview Type of Article

In [ ]:
data.groupby("article_genre").size()

In [ ]:
data.groupby("survey_data").size()

### Graphic Survey per Year

In [ ]:
surveys_per_year = data[data.survey_year.notna()]

In [ ]:
print("Number of publication per year: \n")
print(surveys_per_year.groupby("article_year").size())

In [ ]:
print("Number of surveys per year: \n")
print(surveys_per_year.groupby("survey_year").size())

In [ ]:
print("Average difference between publication and survey year: \n")
print(surveys_per_year.article_year.mean()-surveys_per_year.survey_year.mean())

### Overview of Journals and Proceedings

In [ ]:
print("Publications per journal: \n")
print(data.groupby("article_journal").size().sort_values(ascending=False))

In [ ]:
print("Publications per proceeding: \n")
print(data.groupby("article_proceeding").size().sort_values(ascending=False))

### Overview of authors

In [ ]:
print("Publications per author: \n")
article_authors = pd.DataFrame({"author":[j for i in metadata["article_author"].str.split("; ") for j in i]})
publication_per_author = article_authors.groupby("author").size().sort_values(ascending = False)
print(publication_per_author.head(40))

### Overview of Countries

In [ ]:
# Prepare countries for analyses and visualisation
survey_countries = data.survey_country.values.tolist()
countries_to_add = ["China", "USA", "Spain", "Mexico","Spain", "Mexico", "UK", "Australia", "Canada", "Norway"]
countries_to_drop = ["European Union Member and Accession Countries","['China', 'USA']", "['Spain', 'Mexico']",
                     "['UK', 'Australia', 'Canda']", "Spanish Speaking Countries", "Norwegia"]  
countries_complete = survey_countries + countries_to_add 
countries_for_count = [country for country in countries_complete if country not in countries_to_drop]
country_count = pd.DataFrame({"Country":countries_for_count}).groupby("Country").size()
country_count_df = country_count.reset_index().rename(columns = {0: "Count"})
country_count_df.replace({"USA": "United States of America",
                          "UK": " United Kingdom"}, inplace = True)

In [ ]:
print("Surveys per country: \n")
country_count_df.sort_values(by = "Count", ascending=False)

### Visualise Surveys per Country

In [ ]:
# Create visualisation dataset
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
mapped = world.merge(country_count_df, how="left", left_on=['name'], right_on=['Country'])
mapped.fillna(0, inplace = True)
mapped_without_antarctica = mapped[mapped.continent != "Antarctica"]

In [ ]:
# Create color map
cmap = cm.get_cmap("viridis")
cmap.set_under(color = 'lightgrey')

In [ ]:
# Create visualisation
fig, ax = plt.subplots(1, 1)

divider = make_axes_locatable(ax)

cax = divider.append_axes("right", size="5%", pad=0.1)

ax.set_axis_off();

plot_world = mapped_without_antarctica.plot(column='Count',ax=ax,cax = cax, legend=True, cmap=cmap, vmin= 1)
                                                        
plot_world.figure.set_figwidth(17.5)
plot_world.figure.set_figheight(7.5)

plot_world.figure.savefig('../reports/figures/surveys_per_country.png', dpi=600)

### Overview Survey Operator

In [ ]:
print("Surveys per operator: \n")
print(data.groupby("survey_operator").size().sort_values(ascending = False))

## Identify Research Topics

In this section we will identify the research topics, that we will describe in section 3.1 in the article.

In [ ]:
data_quality_indicator = [x for x in data.columns.tolist() if re.match("mobile_(?!participant|design).*", x)] 
data["article_topic_data_quality"] = data[data_quality_indicator].notna().any(axis = 1)
data["article_topic_data_quality"].sum()

In [ ]:
mobile_participants_indicator = [x for x in data.columns.tolist() if re.match("mobile_participant.*", x)] 
data["article_topic_mobile_participants"] = data[mobile_participants_indicator].notna().any(axis = 1)
data["article_topic_mobile_participants"].sum()

In [ ]:
mobile_design_indicator = [x for x in data.columns.tolist() if re.match("mobile_design.*", x)] 
data["article_topic_mobile_design"] = data[mobile_design_indicator].notna().any(axis = 1)
data["article_topic_mobile_design"].sum()

In [ ]:
alternative_input_indicator = [x for x in data.columns.tolist() if re.match("input_alternative.*", x)] 
data["article_topic_alternative_input"] = data[alternative_input_indicator].notna().any(axis = 1)
data["article_topic_alternative_input"].sum()

In [ ]:
invitation_mode_indicator = [x for x in data.columns.tolist() if re.match("invitation_mode.*", x)] 
data["article_topic_invitation_mode"] = data[invitation_mode_indicator].notna().any(axis = 1)
data["article_topic_invitation_mode"].sum()

### Visualise Publications per Year per Topic

In [ ]:
topic_dic = {"Data Quality": "article_topic_data_quality",
             "Mobile Participants": "article_topic_mobile_participants",
             "Mobile Design": "article_topic_mobile_design",
             "Input Alternative": "article_topic_alternative_input",
             "Invitation Mode": "article_topic_invitation_mode"}

In [ ]:
# Prepare dataset
category_per_year = create_visualisation_data(data, topic_dic)
visualization_year_category = category_per_year.sort_values("Topic").reset_index(drop = True).dropna()

In [ ]:
# Visualise data
sns.set_style('white')
sns.set_context('paper', font_scale = 2)
plot = sns.displot(data=visualization_year_category, x= "Year",bins = 14,
            hue="Topic", col = "Topic", col_wrap=3,legend = False, 
           )
plt.subplots_adjust(wspace=0.2)
plot.set_xticklabels(rotation=45)
plot.set(xlim=(2009,2021),
         xticks= (range(2009,2022)),
         xticklabels=(range(2009,2022)))
plot.savefig('../reports/figures/publications_per_year_per_categories.png')

## Survey Quality

This section prepares all inforamtion showcased in section 3.2 of the article.

### Data Quality

In [ ]:
quality_indicator = find_columns(data, "mobile_(?!participant|design).*")

In [ ]:
know_bias_dic = {"Known Manipulation Bias" : ['mobile_bias_image_presentation',
                                              'mobile_bias_low_versus_high_frequency',
                                              'mobile_bias_navigation_unwillingness',
                                              'mobile_bias_need_for_scrolling',
                                              'mobile_bias_single_versus_multiple_questions_on_page',
                                              'mobile_bias_text_answer_avoiding',
                                              'mobile_joint_versus_seperate_evaluation_bias_more']}

In [ ]:
completion_rate_dic = {"Completion Rate" : ['mobile_completion_rate_close_ended_lower',
                                            'mobile_completion_rate_lower',
                                            'mobile_completion_rate_matrix_lower',
                                            'mobile_completion_rate_open_questions_lower']}

In [ ]:
completion_time_dic = {"Completion Time" : ['mobile_completion_time_breakdown',
                                            'mobile_completion_time_higher',
                                            'mobile_completion_time_higher_subjective',
                                            'mobile_completion_time_lower',
                                            'mobile_completion_time_multiple_questions_higher',
                                            'mobile_completion_time_text_anwser_higher',
                                            'mobile_network_latency_higher',
                                            'mobile_processsing_time_higher',
                                            'mobile_scrolling_time_more',
                                            'mobile_transmission_time_higher']}

In [ ]:
device_impact_dic = {"Mobile Device Impact": ['mobile_device_less_powerful_deivce_higher_processing_time_than_more_powerful',
                                              'mobile_device_less_powerful_deivce_higher_transmission_time_than_more_powerful',
                                              'mobile_device_less_powerful_device_higher_completion_time_than_more_powerful',
                                              'mobile_device_more_powerful_device_higher_completion_time',
                                              'mobile_device_more_powerful_higher_processing_time',
                                              'mobile_device_more_powerful_higher_transmission_time',
                                              'mobile_device_type_influence_breakoff_rate']}

In [ ]:
sensitive_questions_dic = {"Sensitive Questions" : ['mobile_sensitive_questions_answering_less',
                                                    'mobile_sensitive_questions_answering_more',
                                                    'mobile_sensitive_questions_less_sensitive',
                                                    'mobile_sensitive_questions_less_uneasy',
                                                    'mobile_sensitive_questions_more_uneasy',
                                                    'mobile_sensitivity_perceived_higher']}

In [ ]:
primacy_effect_dic = {"Primacy effect" : ['mobile_primacy_effect_higher',
                                          'mobile_primacy_effect_higher_effect_categorical', 
                                          'mobile_response_order_bias_more']}

In [ ]:
avoid_text_dic = {"Text answer avoiding": ['mobile_text_answer_avoid_more']}

In [ ]:
text_length_dic = {"Text answer length" : ['mobile_text_answer_shorter',
                                           'mobile_text_anwser_longer',
                                           'mobile_unaceptably_short_anwser_less']}

In [ ]:
response_rate_dic = {"Reponse rate" : ['mobile_response_rate_complex_question_higher',
                                       'mobile_response_rate_lower']}

In [ ]:
straightlining_dic = {"Straightlining" : ['mobile_straightlining_less',
                                          'mobile_straightlining_more']}

In [ ]:
non_substantial_dic = {"Non substantial Anwser" : ['mobile_non_differentiated_answers_more',
                                                   'mobile_non_substantial_anwser_more',
                                                   'mobile_non_substantial_anwser_single_choice_more',
                                                   'mobile_text_response_nonrelevant_more']}

In [ ]:
scale_effects_dic = {"Scale effects" : ['mobile_scale_effects_more',
                                        'mobile_scale_reliability_lower']}

In [ ]:
breakoff_rate_dic = {"Breakoff rate" : ['mobile_breakoff_rate_higher']}

In [ ]:
usability_dic = {"Usability" : [ 'mobile_technical_difficulties_more',
                                'mobile_usability_lower']}

In [ ]:
survey_satisfaction_dic = {"Survey Satisfaction" : ['mobile_satisfaction_lower',
                                                    'mobile_enjoyment_of_survey_less']}

In [ ]:
socially_undesirable_dic = {"Socially Undesirable Anwsers" : ['mobile_socially_undesirable_answers_more']}

In [ ]:
satisficing_dic = {"Satisficing" : ['mobile_left_aligned_options_choice_less',
                                    'mobile_midpoint_responding_more', 
                                    'mobile_left_aligned_options_choice_less',
                                    'mobile_midpoint_responding_more',
                                    'mobile_more_heaping',
                                    'mobile_motivated_underreporting_more',
                                    'mobile_positive_evaluation_anwser_less',
                                    'mobile_satisficing_check_box_more',
                                    'mobile_instruction_manipulation_check_failure_more',
                                    'mobile_inter_item_correlation_grid_more', 
                                    'mobile_check_all_that_apply_anwsers_less',
                                    'mobile_acquiescence_more']}

In [ ]:
missing_item_dic={"Mising items rate": ['mobile_missing_items_rate_higher']}

In [ ]:
incosistent_anwsers_dic = {"Incosistent,errenous and random Anwsers" : ['mobile_anwser_changes_more',
                                                                        'mobile_construct_validity',
                                                                        'mobile_inconsistent_anwsers_more',
                                                                        'mobile_reponse_randomness_higher',
                                                                        'mobile_error_rate_higher',
                                                                        'mobile_estimates_biased_more',
                                                                        'mobile_measurement_quality_lower']}

In [ ]:
response_time_dic = {"Response time" : ['mobile_response_later_on_day',
                                        'mobile_response_time_faster']}

In [ ]:
incentive_dic = {"Incentive": ["mobile_higher_survey_incentive_higher_participation"]}

In [ ]:
other_dic = {"Other" : ['mobile_results_different',
                        'mobile_willingness_rate_lower', 
                        'mobile_higher_survey_incentive_higher_participation', 
                        'mobile_lost_focus_more',
                        'mobile_low_familarity_completion_time_higher',
                        'mobile_order_by_click_different_results']}

In [ ]:
quality_dic_list = [completion_rate_dic, know_bias_dic, completion_rate_dic, completion_time_dic, device_impact_dic,
                    sensitive_questions_dic, primacy_effect_dic, avoid_text_dic,text_length_dic, response_rate_dic,
                    straightlining_dic,non_substantial_dic, scale_effects_dic, breakoff_rate_dic, usability_dic,
                    survey_satisfaction_dic, socially_undesirable_dic,satisficing_dic, missing_item_dic,
                     incosistent_anwsers_dic, response_time_dic, incentive_dic, other_dic,]

In [ ]:
check_columns(quality_indicator, quality_dic_list)

In [ ]:
# Final citation used for article
design_citation_key_dic = find_citation_key_topics(data, quality_dic_list)

### Survey Participants

In [ ]:
participants_indicator = find_columns(data, "mobile_participants_.*")

In [ ]:
coverage_bias_dic = {"Coverage Bias" : ['mobile_participants_coverage_bias',
                                        'mobile_participants_coverage_bias_weighting_elimated',
                                        'mobile_participants_representative_not',
                                        'mobile_participants_mobile_increasing',
                                        'mobile_participants_insights',
                                        'mobile_participants_increasing_usage_of_smartphone']}

In [ ]:
education_dic = {"Education" : ['mobile_participants_education_higher',
                                'mobile_participants_education_lower',
                                'mobile_participants_higher_educated_willingness_use_mobile_higher',
                                'mobile_participants_higher_education_use_mobile_more',
                                'mobile_participants_more_educated_use_smartphone_more']}

In [ ]:
income_dic = {"Income" : ['mobile_participants_income_higher',
                          'mobile_participants_income_lower',
                          'mobile_participants_social_class_higher',
                          'mobile_participants_lower_income_use_mobile_more',
                          'mobile_participants_higher_income_willingness_use_mobile_higher']}

In [ ]:
gender_dic = {"Gender" : ['mobile_participants_more_female',
                          'mobile_participants_more_male',
                          'mobile_participants_female_willingness_use_mobile_higher',
                          'mobile_participants_female_use_mobile_more',
                          'mobile_participants_male_more_mobile_than_female ',
                          'mobile_participants_male_more_android_than_female',
                          'mobile_participants_male_more_ios_than_female']}

In [ ]:
age_dic = {"Age" : ['mobile_participants_younger', 
                    'mobile_participants_millenial_willing_to_use_emoji',
                    'mobile_participants_millenials_participate_more_mobile',
                    'mobile_participants_younger_willingness_use_mobile_higher',
                    'mobile_participants_not_accepting_mobile_survey_older',
                    'mobile_participants_young_use_mobile_more']}

In [ ]:
urban_dic = {"Urbanity" : ['mobile_participants_more_urban',
                           'mobile_participants_more_urban_use_mobile_more']}

In [ ]:
distraction_dic = {"Distraction" : ['mobile_participants_more_distracted',
                                    'mobile_participants_more_multitasking',
                                    'mobile_participants_multiple_browser_sessions_more',
                                    'mobile_participants_less_alone',
                                    'mobile_participants_more_stranger_present']}

In [ ]:
mobility_dic = {"Mobility of participants": ['mobile_participants_more_mobile',
                                             'mobile_participants_more_moving']}

In [ ]:
device_use_dic = {"Device use" : ['mobile_participants_male_more_android_than_female',
                                  'mobile_participants_male_more_ios_than_female']}

In [ ]:
other_dic = {"Other" : ['mobile_participants_positive_asylum_decision_higher_participation',
                        'mobile_participants_more_willing_passive_data_collection',
                        'mobile_participants_less_trust_in_confidentiality',
                        'mobile_participants_more_progressive']}

In [ ]:
participant_dict_list = [coverage_bias_dic,education_dic,income_dic, gender_dic, age_dic, urban_dic,
                         distraction_dic, mobility_dic, device_use_dic, other_dic ]

In [ ]:
check_columns(participants_indicator, participant_dict_list)

In [ ]:
# Final citation used for article
participants_citation_key_dic = find_citation_key_topics(data, participant_dict_list)

## Survey Design

In this section we prepare the analyses of section 3.3 in the work.

### Mobile Design

In [ ]:
design_dimensions = find_columns(data, "mobile_design_.*")

In [ ]:
emoji_dic = {"Emoji" : ['mobile_design_emoji_different_mean_rating',
                        'mobile_design_emoji_different_satisfaction',
                        'mobile_design_emoji_preference_differs_between_countries',
                        'mobile_design_emoji_request_decreases_completion_time',
                        'mobile_design_emoji_request_increase_informatio_conveyed',
                        'mobile_design_emoji_request_less_complex_survey',
                        'mobile_design_emoji_usage_differs_between_countries']}

In [ ]:
grid_dic = {"Question Grid" : ['mobile_design_grid_versus_item_by_item_anwser_change_more',
                               'mobile_design_grid_versus_item_by_item_completion_time_shorter',
                               'mobile_design_grid_versus_item_by_item_correlation_more',
                               'mobile_design_grid_versus_item_by_item_failure_instruction_manipulation_check_more',
                               'mobile_design_grid_versus_item_by_item_lost_focus_less',
                               'mobile_design_grid_versus_item_by_item_non_differentiation_more',
                               'mobile_design_grid_versus_item_by_item_non_substantial_anwsers_more',
                               'mobile_design_grid_versus_item_by_item_straightlining_more',
                               'mobile_design_grid_versus_item_by_item_subjective_evaluation_lower',
                               'mobile_design_grid_versus_item_by_item_technical_difficulties_more',
                               'mobile_design_split_grid_into_different_questions_reduces_non_differentiation',
                               'mobile_design_more_correlation_in_grid_versus_one_at_a_time_presentation']}

In [ ]:
item_by_item_dic = {"Item by Item" : ['mobile_design_grid_versus_item_by_item_anwser_change_more',
                                      'mobile_design_grid_versus_item_by_item_completion_time_shorter',
                                      'mobile_design_grid_versus_item_by_item_correlation_more',
                                      'mobile_design_grid_versus_item_by_item_failure_instruction_manipulation_check_more',
                                      'mobile_design_grid_versus_item_by_item_lost_focus_less',
                                      'mobile_design_grid_versus_item_by_item_non_differentiation_more',
                                      'mobile_design_grid_versus_item_by_item_non_substantial_anwsers_more',
                                      'mobile_design_grid_versus_item_by_item_straightlining_more',
                                      'mobile_design_grid_versus_item_by_item_subjective_evaluation_lower',
                                      'mobile_design_grid_versus_item_by_item_technical_difficulties_more','mobile_design_item_by_item_higher_number_of_columns_higher_straightlining',
                                      'mobile_design_item_by_item_higher_number_of_columns_higher_survey_rating',
                                      'mobile_design_item_by_item_higher_number_of_rows_higher_dropout',
                                      'mobile_design_item_by_item_higher_number_of_rows_higher_nonreponse',
                                      'mobile_design_item_by_item_higher_number_of_rows_higher_straightlining',
                                      'mobile_design_item_by_item_higher_number_of_rows_higher_survey_rating',
                                      'mobile_design_item_by_item_higher_number_of_columns_higher_dropout',
                                      'mobile_design_more_correlation_in_grid_versus_one_at_a_time_presentation',
                                      'mobile_design_split_grid_into_different_questions_reduces_non_differentiation']}

In [ ]:
paging_dic = {"paging" : ['mobile_design_paging_higher_break_off_rate_than_scrolling',
                          'mobile_design_paging_higher_item_nonresponse_than_scrolling',
                          'mobile_design_paging_higher_primacy_effect_than_scrolling',
                          'mobile_design_paging_longer_completion_time_subjective_than_scrolling',
                          'mobile_design_paging_longer_completion_time_than_scrolling',
                          'mobile_design_paging_lower_item_nonreponse_than_scrolling',
                          'mobile_design_paging_lower_satisfaction_than_scrolling',
                          'mobile_design_paging_more_difficult_than_scrolling']}

In [ ]:
slider_dic = {"Slider" : ['mobile_design_slider_anwser_less_accurate',
                          'mobile_design_slider_satisficing_higher',
                          'mobile_design_sliders_more_negative_evaluation_than_buttons',
                          'mobile_design_sliders_more_non_response_than_buttons',
                          'mobile_design_sliders_more_non_substantive_anwser_than_buttons']}

In [ ]:
visual_analogue_dic = {"Visual Analogue" : ['mobile_design_visual_analogue_scales_more_negative_evaluation_than_buttons',
                                            'mobile_design_visual_analogue_scales_more_non_response_than_buttons',
                                            'mobile_design_visual_analogue_scales_more_non_substantive_anwser_than_buttons']}

In [ ]:
mobile_optimisation_impact_dic = {"Mobile optimisation impact" : ['mobile_design_not_optimized_mobile_avoid_text_answer_more_than_optimized',
                                                                  'mobile_design_not_optimized_mobile_completion_time_higher_than_optimized',
                                                                  'mobile_design_not_optimized_mobile_less_text_than_optimized',
                                                                  'mobile_design_not_optimized_mobile_non_substantial_anwser_more_than_optimized',
                                                                  'mobile_design_optimized_online_higher_completion_rate']}

In [ ]:
navigation_dic = {"(Automatic) Navigation with Text, buttons or icons" : ['mobile_design_always_on_next_button_data_quality_difference_to_two_columns',
                                                                          'mobile_design_icon_navigation_versus_text_navigation_navigation_errors_more',
                                                                          'mobile_design_icon_navigation_versus_text_navigation_reponse_time_first_try_slower',
                                                                          'mobile_design_icon_navigation_versus_text_navigation_satisfaction_higher',
                                                                          'mobile_design_next_button_always_visible_less_missing_data_than_not_always_visible',
                                                                          'mobile_design_next_button_always_visible_lower_completion_time_than_not_always_visible',
                                                                          'mobile_design_next_button_always_visible_more_missing-item',
                                                                          'mobile_design_next_button_always_visible_shorter_completion_time',
                                                                          'mobile_design_next_button_always_visible_with_columns_less_missing_data_than_not_always_visible',
                                                                          'mobile_design_optimized_and_auto_forwarding_higher_satisfaction',
                                                                          'mobile_design_optimized_and_auto_forwarding_shorter_reponse_time',
                                                                          'mobile_design_text_navigation_preferred_over_icon_navigation',
                                                                          'mobile_design_text_navigation_preferred_over_icon_navigation',
                                                                          'mobile_design_two_columns_with_next_button_always_visible_less_missing_data_than_one_column'
                                                                         ]}

In [ ]:
modularisation_dic = {"Modularisation" : ['mobile_design_modularization_increases_mobile_participation',
                                          'mobile_design_questions_all_at_once_higher_completion_time',
                                          'mobile_design_questions_all_at_once_higher_usability',
                                          'mobile_design_questions_by_one_at_a_time_higher_completion_time',
                                          'mobile_design_questions_by_one_at_a_time_higher_usability',
                                          'mobile_design_questions_by_section_higher_completion_time',
                                          'mobile_design_questions_by_section_higher_usability',
                                          'mobile_design_recommendations',
                                          'mobile_design_split_questionnaire_design_more_accurate']}

In [ ]:
vertical_versus_horizontal_dic = {"Vertical versus horizontal" : ['mobile_design_vertical_higher_primacy_effect_than_horizontal',
                                                                  'mobile_design_vertical_longer_completion_time_than_horizontal',
                                                                  'mobile_design_vertical_more_difficult_than_horizontal',
                                                                  'mobile_design_layout_vertical_lower_item_nonresponse_than_horizontal']}

In [ ]:
radio_button_dic = {"Radio button" : ['mobile_design_android_spinner_higher_satisfaction_than_radio_button',
                                      'mobile_design_android_spinner_longer_completion_time_than_radio_button',
                                      'mobile_design_android_spinner_lower_accuracy_than_radio_button',
                                      'mobile_design_ios_picker_longer_completion_time_than_radio_button',
                                      'mobile_design_ios_picker_lower_accuracy_than_radio_button',
                                      'mobile_design_ios_picker_versus_radio_button_repondent_burden_higher',
                                      'mobile_design_ios_picker_versus_radio_button_satisfaction_higher',
                                      'mobile_design_no_clear_effects_of_different_radio_button_alternatives']}

In [ ]:
multimedia_dic = {"Image and Video" : ['mobile_design_video_able_to_view_less',
                                       'mobile_design_image_effect_lower']}

In [ ]:
other_dic = {"Other" : ['mobile_design_standards',
                        'mobile_design_agree_disagree_shorter_completion_time_than_item_specific_question',
                        'mobile_design_date_picker_anwser_less_accurate',
                        'mobile_design_date_picker_less_successful_answers',
                        'mobile_design_less_item_scrolling_shorter_completion_time',
                        'mobile_design_less_items_scrolling_higher_breakoff',
                        'mobile_design_measurement_invariance_decremental_scale',
                        'mobile_design_measurement_invariance_incremental_scale',
                        'mobile_design_numeric_label_smaller_impact_than_pc',
                        'mobile_design_one_column_data_quality_difference_to_two_columns']}

In [ ]:
design_dict_list = [emoji_dic, grid_dic,item_by_item_dic,paging_dic, slider_dic, visual_analogue_dic,
                    mobile_optimisation_impact_dic, navigation_dic, modularisation_dic, 
                    vertical_versus_horizontal_dic, radio_button_dic, multimedia_dic, other_dic]

In [ ]:
check_columns(design_dimensions, design_dict_list)

In [ ]:
# Final citation used for article
design_citation_key_dic = find_citation_key_topics(data, design_dict_list)

### Invitation Mode

In [ ]:
invitation_indicator = find_columns(data, "invitation_mode_.*")

In [ ]:
sms_dic = {"SMS" : ['invitation_mode_breakoff_rate_sms_invitation_higher',
                    'invitation_mode_mobile_participants_rate_sms_invitation_higher',
                    'invitation_mode_mobile_reponse_rate_sms_invitation_higher',
                    'invitation_mode_mobile_reponse_time_after_reminder_faster',
                    'invitation_mode_mobile_reponse_time_sms_invitation_shorter_than_e_mail_invitation',
                    'invitation_mode_rdd_sms_response_rate_low',
                    'invitation_mode_sms_inivitation_higher_mobile_participation',
                    'invitation_mode_sms_invitation_mobile_breakoff_rate_higher']}

In [ ]:
mail_dic = {"Mail" : ['invitation_mode_optimized_letter_changes_age_structure',
                      'invitation_mode_optimized_letter_means_higher_mobile_reponses',
                      'invitation_mode_optimized_letter_means_higher_reponses']}

In [ ]:
invitation_dict_list = [sms_dic, mail_dic]

In [ ]:
check_columns(invitation_indicator, invitation_dict_list)

In [ ]:
# Final citation used for article
invitation_citation_key_dic = find_citation_key_topics(data, invitation_dict_list)

### Alternative Input

In [ ]:
alternative_input_indicator = find_columns(data, "input_alternativ.*")

In [ ]:
voice_dic = {"Voice" : ['input_alternative_android_voice_higher_nonreponse_than_text',
                        'input_alternative_android_voice_less_valid_answers_than_text',
                        'input_alternative_android_voice_more_characters_than_text',
                        'input_alternative_android_voice_more_complicated_than_text',
                        'input_alternative_android_voice_more_techical_problems_than_text',
                        'input_alternative_android_voice_trust_anonymity_less_than_text',
                        'input_alternative_ios_voice_higher_nonreponse_than_text',
                        'input_alternative_ios_voice_less_characters_than_text',
                        'input_alternative_ios_voice_less_valid_answers_than_text',
                        'input_alternative_ios_voice_more_complicated_than_text',
                        'input_alternative_ios_voice_trust_anonymity_less_than_text',
                        'input_alternative_voice_different_instruction_reduce_technical_problems',
                        'input_alternative_voice_faster_per_character_than_text']}

In [ ]:
image_dic = {"Image" : ['input_alternative_image_taking_lower_compliance_than_image_uploading',
                        'input_alternative_not_clear_reasons_for_non_compliance_images']}

In [ ]:
messenger_dic = {"Messenger": ['input_alternative_messenger_completion_rate_lower',
                               'input_alternative_messenger_completion_time_longer',
                               'input_alternative_messenger_shorter_text',]}

In [ ]:
sensor_data_dic = {"Sensor data": ['input_alternative_moving_data_higher_motion_means_higher_completion_time_if_grid',
                                   'input_alternative_moving_data_higher_motion_means_higher_completion_time_if_single_question',
                                   'input_alternative_moving_data_higher_motion_means_higher_primacy_effect_if_grid',
                                   'input_alternative_moving_data_higher_motion_means_higher_primacy_effect_if_single_question',
                                   'input_alternative_moving_data_standing_less_extreme_values_than_walking',
                                   'input_alternative_moving_data_standing_less_primacy_more_than_walking',
                                   'input_alternative_moving_data_standing_more_middle_values_than_walking',
                                   'input_alternative_moving_data_standing_more_recency_than_walking',
                                   'input_alternative_moving_data_standing_motion_compliance_more_than_walking']}

In [ ]:
alternative_input_dict_list = [voice_dic, image_dic, messenger_dic, sensor_data_dic]

In [ ]:
check_columns(alternative_input_indicator, alternative_input_dict_list)

In [ ]:
# Final citation used for article
alternative_input_citation_key_dic = find_citation_key_topics(data, alternative_input_dict_list)

### Other topics

In [ ]:
other_topics = find_columns(data, "^(?!mobile|input|invitation|online|paradata).*")

This is the end of the file. Further analyses not presented in the paper, can be found in the notebok 2.0-jl-qualitative-meta-analysis-of-research-results.